In [14]:
import os

import SimpleITK as sitk
import importlib

import helpers

importlib.reload(helpers)
from helpers import explore_3D_data, explore_3D_data_comparison, explore_3D_data_with_mask
from preprocessing.filter_methods import N4_bias_filter


In [15]:
root_dir = "dataset"

t2_file = None

In [16]:

for root, dirs, files in os.walk(root_dir):
    print("found root directory: ", root)
    for f in files:
        if f.endswith("_T2w.nii.gz"):
            t2_file = os.path.join(root, f)
            break
    if t2_file:
        break

found root directory:  dataset
found root directory:  dataset\binary_epvs_groundtruth
found root directory:  dataset\binary_epvs_groundtruth\mask
found root directory:  dataset\binary_epvs_groundtruth\mask\sub-001
found root directory:  dataset\binary_epvs_groundtruth\mask\sub-002
found root directory:  dataset\binary_epvs_groundtruth\mask\sub-003
found root directory:  dataset\binary_epvs_groundtruth\mask\sub-004
found root directory:  dataset\binary_epvs_groundtruth\mask\sub-005
found root directory:  dataset\binary_epvs_groundtruth\mask\sub-006
found root directory:  dataset\binary_epvs_groundtruth\mask\sub-007
found root directory:  dataset\binary_epvs_groundtruth\mask\sub-008
found root directory:  dataset\binary_epvs_groundtruth\mask\sub-009
found root directory:  dataset\binary_epvs_groundtruth\mask\sub-010
found root directory:  dataset\binary_epvs_groundtruth\mask\sub-011
found root directory:  dataset\binary_epvs_groundtruth\mask\sub-012
found root directory:  dataset\binary_

In [17]:

if t2_file:
    print("Found T2W: ", t2_file)
    raw_img_sitk = sitk.ReadImage(t2_file)
    raw_img_sitk =  sitk.DICOMOrient(raw_img_sitk,'RPS')
    raw_img_sitk_arr = sitk.GetArrayFromImage(raw_img_sitk)

else:
    print("No T2W found")

Found T2W:  dataset\braindata\sub-001\anat\sub-001_T2w.nii.gz


In [18]:
explore_3D_data(raw_img_sitk_arr, cmap="gray")

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

In [19]:
raw_img_sitk = sitk.ReadImage("vesselness_output.nii.gz")
raw_img_sitk = sitk.DICOMOrient(raw_img_sitk, 'RPS')
raw_img_sitk_arr = sitk.GetArrayFromImage(raw_img_sitk)

explore_3D_data(raw_img_sitk_arr, cmap="grey")

RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: D:\a\SimpleITK\SimpleITK\Code\IO\src\sitkImageReaderBase.cxx:91:
sitk::ERROR: The file "vesselness_output.nii.gz" does not exist.

In [24]:
n4_filt_img = N4_bias_filter(raw_img_sitk)

In [25]:
explore_3D_data_comparison(arr_before=raw_img_sitk_arr,
                           arr_after=sitk.GetArrayFromImage(n4_filt_img),
                           cmap="grey"
                           )

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

In [20]:
img_mask = sitk.ReadImage("vesselness_output.nii.gz", sitk.sitkFloat32)
img_mask = sitk.DICOMOrient(img_mask,'RPS')
masked_arr = sitk.GetArrayFromImage(img_mask)

raw_img_sitk = sitk.ReadImage("dataset/braindata/binary_epvs_groundtruth/mask/sub-001/sub-001_desc-mask_PVS.nii.gz", sitk.sitkFloat32)
raw_img_sitk =  sitk.DICOMOrient(raw_img_sitk,'RPS')
raw_img_sitk_arr = sitk.GetArrayFromImage(raw_img_sitk)

explore_3D_data_with_mask(
    arr=raw_img_sitk_arr,
    mask=masked_arr,
    alpha=0.8,
    cmap="grey"
)



RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: D:\a\SimpleITK\SimpleITK\Code\IO\src\sitkImageReaderBase.cxx:91:
sitk::ERROR: The file "vesselness_output.nii.gz" does not exist.

In [38]:
import pandas as pd

df = pd.read_csv("dataset/HUNT3&4_MRI_Sex_age.csv", header=0)

print(df.head(2))


  HUNT4 numbers;HUNT3 numbers
0         10215;9410000010908
1         23318;9410000010922


In [42]:
import pandas as pd

# Read CSVs with the correct delimiter
file1 = pd.read_csv("dataset/HUNT3&4_MRI_Sex_age.csv", sep=";")
file2 = pd.read_csv("dataset/VRS_Numbers.csv", sep=";")

# Strip any leading/trailing spaces in column names
file1.columns = file1.columns.str.strip()
file2.columns = file2.columns.str.strip()

# Check column names
print("File1 columns:", file1.columns.tolist())
print("File2 columns:", file2.columns.tolist())

# Merge on the HUNT3 column (after stripping spaces)
merged = pd.merge(file1, file2, on="HUNT3 numbers", how="inner")

# Save the merged CSV
merged.to_csv("merged_file.csv", index=False, sep=";")  # keep semicolon separator
print("Merged file saved as 'merged_file.csv'")


File1 columns: ['HUNT4 numbers', 'HUNT3 numbers']
File2 columns: ['Kjønn_m1_k2', 'Ny_beregnet_alder', 'HUNT3 numbers', 'Bg_VH', 'Cso_VH', 'Gruppe_BG', 'Gruppe_CSO', 'TOTALvariabel', 'TOTALgruppe']
Merged file saved as 'merged_file.csv'
